In [28]:
import gzip, shutil
import os 
import data_utils as du 
import tensorflow as tf 
from tensorflow import keras 
import numpy as np 
from six.moves import cPickle as pickle
import numpy as np
import os
from matplotlib.pyplot import imread
import matplotlib.pyplot as plt 
import platform
import data_utils as du 
from cnn_model import *
from solver import Solver
import qunatize_resnet20 as q_res
import quantize_layers as ql
import tensorflow_hub as hub 
import quantize_util as qu 
import quantize_complier as qc 


%load_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext autoreload
plt.rcParams['figure.figsize'] = (15.0, 15.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

Data_folder = 'CIFAR10_Data'
Data_fn = 'cifar-10-python'
X_train, y_train, X_val, y_val, X_test, y_test = du.get_CIFAR10_data(subtract_mean =True)
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int64
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [29]:
data_set = {
    'train_data': X_train,
    'train_label': y_train,
    'val_data':X_val,
    'val_label':y_val,
}
train_options = {
    'optimizer': 'adam',
    'lr': 0.001,
    'epoch_num': 1,
    'from_logits':True,
    'save_dir': 'resnet20_model'
}
model_path = 'resnet20_model/trained_15'
full_model = hub.KerasLayer(model_path,trainable=True)

qunatizer = ql.ModelQunatize() 
model_q = q_res.Resnet20(qunatizer)
qu.copy_weight(model_q,full_model)
q_layers = model_q.get_quantizable_layers()
for layer in q_layers:
    print(layer)

solver = Solver(model_q,data_set,train_options,save_model=False)

data_shuffler = qc.DataShuffler(X_train, y_train,sample_size=1024)


def train_fn(model,solver=solver):
    solver = Solver(model,data_set,train_options,
                    save_model=False,verbose=False,
                    plot_graph=False)
    return solver.train(iter=int(5000/32))

def test_fn(model,solver=solver,shuffle=False):
    solver = Solver(model,data_set,save_model=False,
                    verbose=False,plot_graph=False)
    #X_s,y_s = data_shuffler.sample(shuffle)
    #return solver.test_model(X_test[0:8*1024-1],y_test[0:8*1024-1])
    #return solver.test_model(X_test[0:1024*6-1],y_test[0:1024*6-1])
    #return solver.test_model(X_s,y_s)
    return solver.test_model(X_test,y_test)






In [30]:
q_complier = qc.QunatizeComplier(model_q,full_model,q_layers,train_fn,test_fn,verbose=True,target_compression=0.12,max_loss=0.03)
q_complier.quantize_model()

5989562497416046e-06,None][3.342025138408644e-06,None][3.3854350931505905e-06,None][1.0850684475371963e-06,None][1.2152725048508728e-06,None][1.7795246094465256e-06,None][-2.1692054374966574e-08,None][-6.510203576226559e-08,None][-1.4105656021001778e-07,None][2.2786359465953865e-07,None][-2.169852209021883e-08,None][-1.1935804877793998e-07,None][-7.052828010500889e-08,None][5.200195573706878e-06,None][5.026584858569549e-06,None][1.2098527122361702e-06,None][1.0579404374766455e-07,None][-4.8828141530066205e-08,None][4.218798039801186e-06,1]]
reduce wl on layer 13 kernel,acc_loss: 0.0, current accuracy:0.6646000146865845, compression rate: (0.1294634538990706, 1111712)
<quantize_complier.WlflList object at 0x7f597575e490>[[6,5,0,0][4,4,0,0][4,4,0,0][3,4,0,0][4,4,0,0][5,5,0,0][4,4,0,0][6,6,0,0][6,6,0,0][5,5,0,0][4,5,0,0][6,6,0,0][8,8,0,0][7,8,0,0][3,5,0,0][3,5,0,0][3,5,0,0][3,8,0,0][5,8,0,0][5,4,0,0]]
<quantize_complier.AccuracyLossPerBitList object at 0x7f599876a4c0>[[1.11111612568493e-0

(<qunatize_resnet20.Resnet20 at 0x7f5a68c187c0>,
 <quantize_complier.WlflList at 0x7f597575e490>)

In [31]:

model_save_path = 'complier_tmp/3deep_compressed_resnet20_model'
wlfl_save_path = 'complier_tmp/3deep_compressed_resnet20_wlfl.txt'
model_q.model.save_weights(model_save_path)
tf.saved_model.save(model_q.model,model_save_path)
q_complier.wl_agent.get_wl_list().save(wlfl_save_path)


INFO:tensorflow:Assets written to: complier_tmp/3deep_compressed_resnet20_model/assets


In [32]:

model_q.model.load_weights(model_save_path)
q_layers = model_q.get_quantizable_layers()
wlfl_list = qc.WlflList(len(q_layers))
wlfl_list.load(wlfl_save_path)
q_complier.fl_agent.apply_wlfl_to_layers(q_layers,wlfl_list)
solver = Solver(model_q,data_set,train_options)
solver.test_model(X_test,y_test)
q_complier.wl_agent.get_compression_rate()


Loss: 1.8155468702316284,Test Accuracy: 66.05999755859375


(0.08921690653112027, 766112)

In [33]:
solver.test_model(X_test,y_test)

Loss: 1.8155468702316284,Test Accuracy: 66.05999755859375


(<tf.Tensor: shape=(), dtype=float32, numpy=1.8155469>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.6606>)